# Описание, заметки, размышления

Для прогнозирования используются данные о покупках юзеров за период 2021 года. Всего доступно больше 2 млн. строк.

Флаг оттока проставляется, если после покупки юзера прошло больше 45 дней.

Поведение юзера описывается историей его покупок, суммой чека и размером скидки к чеку. В компании действует программа лояльности, участвуя в которой юзеры получают скидку на любую покупку и могут получать дополнительные скидки по разным промо-акциями.

Описание данных:
data.csv
* `'clnt_ID'` - уникальный айди юзера, str
* `'timestamp'` - дата и время совершения покупки, datetime
* `'gest_Sum'` - сумма покупки, float
* `'gest_Discount'` - сумма скидки, float

target.csv
* `'clnt_ID'` - уникальный айди юзера, str
* `'target'` - флаг оттока, int: 1 если юзер ушел в отток | 0 если НЕ отток

Это задача бинарной классификации. Используем метрику: `roc auc`.

<!-- ## Что здесь можно сделать? - Заметки 
**Нужно помнить, что по сути это тайм серия.**  -- **Внимательно!**  
Т.е. я предполагаю, что клиент покупает, покупает, покупает, потом фигак и перестал покупать. Один клиент - один вектор.
А тут куча вектором принадлежат одному клиенту. 

Т.е. я бы даже сказал, что в том виде в котором оно есть, модель особо-то ничего не найдет. Как она сможет по дате, сумме покупки и скидке предсказать уйдет человек или нет, не зная его истории?
---ОТВЕТ---
Да, это последовательность событий, но прям как таймсерию использовать не можем, потому что нерегулярные сигналы и из-за этого много сложностей.
Но! Можно использовать эти же строки, но к ним добавлять "память" о клиенте, т.е. его историю к конкретной сделке.

**Что может говорить, что клиент собрался уйти и не вернуться?**  
Т.е. гипотетически что может говорить о том, что клиент собирается уйти и не вернуться?
1. Увеличивается интервал между покупками от покупки к покупке. Типа 1 день, потом 2, потом 5 и т.д.
2. Как-то изменяется объем чека (уменьшается, увеличивается?) 
3. Как-то меняется ассортимент (типа вот он покупал одно и тоже, а тут вдруг перестал покупать одно и тоже (испортился товар на его вкус) и он либо сразу ушел, либо попробовал другое и остался, либо попробовал другое - ему не понравилось - ушел

**Как должен работать предикт? Какие данные должны поступать на вход?**  -- **Есть пол-ответа**  
~~Это видимо тоже относится к тому, что это своего рода таймсерия~~
Вся инфа находится в БД, которая обновляется ночью. И модель будет работать ночью и может использовать всю бд. Как именно пока я не понимаю.


**Если сейчас дать модели на вход просто дату, сумму чека и скидку...**  
То она будет пытаться предсказать отток исключительно по дате, сумме чеку и скидке - врядли у нее что-то получится взразумительное.


**Сумма в месяц стабильна для юзера, а отклонение может указывать на отток**  
По результатам уже проведенного анализа есть такой вывод:
сумма, которую готов тратить юзер в месяц,  достаточна стабильна и слабо меняется со временем. Потратив в этом месяце больше обычного, юзер скорее всего в следующем не будет покупать;

По каждому клиенту взять средний чек, каким-то образом определить порог отклонения от этого среднего чека и посмотреть насколько факт этого отклонения коррелирует с оттоком. Может это будет хорошая фича. Но проблема в том, что для предсказания не достаточно 

**Самый простой и очевидный способ подготовки данных**  
Это взять по каждому клиенту кол-во покупок, min,max,mean,median,stdev по gest_Sum, gest_Discount, и перерывом между покупками

**Вариант еще лучше!**  
Используем каждую строку как есть, не переводим ее в строку по уникальному клиенту, а точно также одна строка одна сделка.
К каждой такой строке мы накопительно добавляем кумулятивную инфу о клиенте, т.е.:
* Сколько дней на момент сделки прошло с момента первой сделки
* Какой на текущий момент средний чек
* Какая разница между среднем чеком и чеком сделки
* Сколько сделок произошло на момент текущей сделки
* Сколько сделок в месяц в среднем



## Вопросы:
**>>>Что значат 0 в gest_sum?<<<**

**Как работают скидки? Это накопительные баллы или просто скидки? Почему основные скидки не больше 100 руб, но бывают аж в 3000 руб.?** -->

Структура:  

0. Описание, заметки, размышления
1. Импорты и настройки
2. Предобработка
3. EDA
4. Feature engineering
5. Baseline and model selection
6. Model tuning

# Импорты и настройки

In [1]:
import warnings

import numpy as np
import pandas as pd

import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns

# from pandas_profiling import ProfileReport

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from catboost import CatBoostClassifier, Pool

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

from sklearn.metrics import f1_score


In [2]:
# Настройки pandas
# pd.set_option('display.max_rows', 100)
pd.options.display.float_format = '{:.3f}'.format

In [3]:
# np set random seed
rand_state = 777
# rng = np.random.default_rng(rand_state)

# Предобработка

In [4]:
df = pd.read_csv('data/data.csv', parse_dates=[1])
df_target = pd.read_csv('data/target.csv')

In [5]:
display(df.head())
df_target.head()

,clnt_ID,timestamp,gest_Sum,gest_Discount
0,193B4268-0B4A-475E-B1D0-FF5515E29D29,2021-01-02 09:09:17.060,900.000,300.000
1,8DA65A37-C1D0-41D4-98E1-AB6C5BF1367F,2021-01-02 09:12:24.850,165.000,55.000
2,26ACF3C8-25C8-4345-ABC2-33DA15EA6454,2021-01-02 09:38:21.643,800.500,25.500
3,0F77DDB3-A9A7-44BE-AAAB-9DF59B66A695,2021-01-02 09:45:17.793,580.000,0.000
4,F16BCF77-FA5A-4093-B7E3-FA86E2B1EA31,2021-01-02 09:59:50.453,148.900,3.100


,clnt_ID,target
0,000070A8-DB9E-4AB7-8C4D-6169D4AEBB2A,1
1,00007EB0-6331-438E-A917-E9840C260876,0
2,0000993D-A30E-4233-AB3F-D368D9A0D2C4,1
3,0000A724-7BC5-408F-9F16-6CC3AB16322F,0
4,0000B90C-56DE-43C2-A213-624AFBE36DB2,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2371635 entries, 0 to 2371634
Data columns (total 4 columns):
 #   Column         Dtype         
---  ------         -----         
 0   clnt_ID        object        
 1   timestamp      datetime64[ns]
 2   gest_Sum       float64       
 3   gest_Discount  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 72.4+ MB


In [7]:
df_target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255109 entries, 0 to 255108
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   clnt_ID  255109 non-null  object
 1   target   255109 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.9+ MB


In [8]:
# Изменим тип данных в timestamp на datetime
# df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S.%f')
# Добавим столбец с только датой.
# df['date'] = df['timestamp'].dt.date

Проверим на дубликаты и пропуски.

In [9]:
df.isna().sum()

clnt_ID          0
timestamp        0
gest_Sum         0
gest_Discount    0
dtype: int64

Пропусков нет.

In [10]:
df.duplicated().sum()

129

Есть дубликаты, посмотрим на них.

In [11]:
df.loc[df.duplicated(keep=False)]

,clnt_ID,timestamp,gest_Sum,gest_Discount
639,1FA367BD-8E07-437E-AB6A-009D53533128,2021-01-02 12:00:00,982.200,0.000
663,1FA367BD-8E07-437E-AB6A-009D53533128,2021-01-02 12:00:00,982.200,0.000
85786,3319362D-F6AD-498B-9B53-F59D0418915E,2021-01-14 14:00:00,1189.000,0.000
85787,3319362D-F6AD-498B-9B53-F59D0418915E,2021-01-14 14:00:00,1189.000,0.000
133279,A145C6F8-EDCB-47B1-BF5D-B328F6AF97CA,2021-01-20 16:00:00,0.000,0.000
...,...,...,...,...
2165277,E99EFF35-01BE-49A9-8F61-641F752051E5,2021-11-08 15:00:00,0.000,0.000
2174627,1A4C0239-A7AB-4188-84DB-8F52FBF6E13C,2021-11-10 11:00:00,282.240,37.760
2174629,1A4C0239-A7AB-4188-84DB-8F52FBF6E13C,2021-11-10 11:00:00,282.240,37.760
2198034,C9D86AF9-373C-4531-92A4-8FC752DD5D75,2021-11-14 12:00:00,0.000,0.000


Удалим дубликаты.

In [12]:
df = df.drop_duplicates()

In [13]:
df.duplicated().sum()

0

Дубликаты удалили.

Проверим кол-во строк, где сумма и скидка равны 0

In [14]:
mask = (df['gest_Sum'] == 0) & (df['gest_Discount'] == 0)
df[mask]

,clnt_ID,timestamp,gest_Sum,gest_Discount
304,CB9F63F8-16DE-4749-A6FB-F5F7FFB946D4,2021-01-02 11:00:00.000,0.000,0.000
309,F2B2579C-F050-4B50-A289-C54FBD1DCB28,2021-01-02 11:00:00.000,0.000,0.000
642,A004C991-A4B5-423B-A339-24FDB99711AD,2021-01-02 12:00:00.000,0.000,0.000
974,F8605C66-5ACA-4EFF-A636-683DD86F042E,2021-01-02 12:35:13.187,0.000,0.000
1545,45382F9D-3EF4-47CF-8E37-BCA426B5DFCD,2021-01-02 13:32:36.507,0.000,0.000
...,...,...,...,...
2371013,F162A21A-6B74-4E09-AB33-5EED5720E54F,2021-12-15 19:39:47.203,0.000,0.000
2371014,80B68B0F-AA0D-4898-AE46-E18CC04A48D9,2021-12-15 19:39:53.083,0.000,0.000
2371306,77454F15-DEDF-483F-B5C9-15F51F1235C4,2021-12-15 20:30:00.000,0.000,0.000
2371350,5637A054-68C2-41AB-BD0C-108034255F3B,2021-12-15 20:37:28.547,0.000,0.000


Будем считать, что это техническая ошибка. Удалим эти строки.

In [15]:
df = df.drop(index=df[mask].index)

In [16]:
df[mask]

C:\Users\SnakeRZR\AppData\Local\Temp\ipykernel_10736\2581737305.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[mask]


,clnt_ID,timestamp,gest_Sum,gest_Discount


Проверим данные на дубликаты по пользователю и времени.

In [17]:
df[df.duplicated(['clnt_ID','timestamp'],keep=False)]

,clnt_ID,timestamp,gest_Sum,gest_Discount
657,94B60CD3-F08B-4BE5-9D8C-892E1A41D987,2021-01-02 12:00:00,1389.000,0.000
658,94B60CD3-F08B-4BE5-9D8C-892E1A41D987,2021-01-02 12:00:00,600.000,0.000
4036,4E292158-5A60-4B49-8DD0-97491A0E1B84,2021-01-02 17:00:00,211.200,0.000
4037,4E292158-5A60-4B49-8DD0-97491A0E1B84,2021-01-02 17:00:00,930.360,0.000
5810,B6A1523B-D7DB-4F00-834F-33069B30F7F9,2021-01-02 21:00:00,552.120,0.000
...,...,...,...,...
2370292,1DFCF34F-F1ED-4B15-BCD2-11FC4BBC710E,2021-12-15 18:00:00,384.000,95.000
2371307,1DFCF34F-F1ED-4B15-BCD2-11FC4BBC710E,2021-12-15 20:30:00,339.000,80.000
2371314,1DFCF34F-F1ED-4B15-BCD2-11FC4BBC710E,2021-12-15 20:30:00,241.500,47.500
2371527,5C5D0F65-AC7E-44BA-807D-9A97F01B8CC6,2021-12-15 21:25:00,215.250,38.750


Объединение этих данных может повлиять на статистики, поэтому оставим как есть.

Отсортируем датасет по времени.

In [ ]:
df = df.sort_values(by='timestamp')

In [18]:
# # Проверим отсортирован ли датасет по дате, сравнив его с отсортированным.
# df_temp = df.sort_values(by='timestamp')
# df.equals(df_temp)
# # Не отсортирован.  
# # Отсортируем датасет по дате.
# df = df.sort_values(by='timestamp')

# EDA

Вопросы к данным?
1. Действительно ли таргет 1 там, где дельта между датами покупки >= 45 дней? И такой ли должен быть таргет?
2. Есть ли дисбаланс классов (по сделкам и по клиентам)
3. Есть ли какие-то аномалии (неожиданные значения) в 
    * датах,
    * сумме чека за вычетом скидки
    * в размере скидки

Сравнение по таргету:  
4. Отличаются ли клиенты, которые ушли в отток от тех, которые не ушли по:  

    * чеку за вычетом скидки,  
    * по скидке,  
    * по общей стоимости (без учета скидки),   
    * по длительности в днях между покупками  
    
5. Корреляции в данных по сделкам и по клиентам

6. Исследование типов клиентов, о которых речь пойдет ниже.

## Проверка таргета

Заявляется, что в отток попадают те, у кого максимальная дельта в днях между покупками хоть раз становится равна или превышает 45 дней.  
Расширим эту логику следующим образом, учтя сочетания следующих вещей:
1. Покупок было сделано 1 или больше
2. При покупках больше 1, был ли хоть раз промежуток в днях между ближайшими покупками >=45?
3. На условный текущий момент с момента последней сделки прошло ли 45 дней?

И введем понятие условного текущего момента, которым будет являться либо последний день года (2021-12-31), либо последняя дата в данных, либо какая-то конкретная дата, по которую делали выгрузку данных (по какой-то причине).

В соответствии с этим, разделим всех клиентов на типы:
1. Те, кто сделал всего одну покупку за весь исследуемый период
2. Те, кто сделал больше 1 покупки
    * **ни разу** промежуток между покупками в днях не становился >= 45
    * на условный текущий момент (2021-12-31) с момента последней сделки не прошло 45 дней
3. Те, кто сделал больше 1 покупки 
    * и максимальная дельта в днях между двумя ближайшими хоть раз становилась >= 45.
    * но с последней сделки не прошло 45 дней.
4. Те, кто сделал больше 1 покупки 
    * максимальная дельта хоть раз становилась >= 45.
    * и к условному текущему моменту (2021-12-31) после последней сделки прошло больше 45 дней
5. Те, кто сделал больше 1 покупки, 
    * максимальная дельта была всегда < 45, 
    * к условному текущему моменту (2021-12-31) после последней сделки прошло больше 45 дней.

Все, кроме типа 2, можно условно отнести к оттоку, но это ли нужно бизнесу? 

**Тип 1**: эти клиенты попробовали продукт, но не стали лояльными, поэтому к ним не стоит задача их *вернуть*. Работа с этим типом, я предполагаю, не вписывается в задачу возврата лояльных клиентов, которые ушли, потому что они изначально не были лояльными.

**Тип 2**: Это лояльный тип клиентов, в отток не ушел. В нем можно поискать статистические признаки их лояльности, сравнив с другими.

**Тип 3**: Это тоже условно лояльный тип клиентов, потому что с точки зрения обыденного представления, вполне вероятно, что лояльный регулярный клиент может пропустить месяц или больше в своих регулярных покупках по причинам, не связанным с бизнесом и продуктом. Ведь он же вернулся.

**Тип 4**: Можно ли этот тип рассматривать, как отток? В их истории уже были длительные перерывы. Это может быть полу-регулярный сбив ритма покупок, т.е. не говорит о потери лояльности, а о личных регулярных причинах, либо это действительно окончательный отток. По личным ли причинам или из-за продукта/бизнеса? - это неизвестно.

**Тип 5**: Можно ли этот тип также рассматривать, как отток? Возможно это отток, а возможно разовая личная причина, но клиент вернется.

У нас здесь вырисовываются две переменные:  
* кол-во дней, в которые были произведены покупки (напр., если в одном дне было сделано больше 1 покупки, то такой день все равно считается за один) 
* дельта в днях между покупками (почему она именно 45, звучит как слишком ровное число. Возможно есть какой-то другой порог, который будет ценнее для предсказаний)

К этим переменным есть два вопроса?  
* Сколько таких дней "покупок" должно быть достаточно, чтобы мы могли по ним описать поведение клиента? Чтобы впоследствии иметь основание определить порог аномального поведения?
* Почему дельта устанавливается в 45 дней? Похоже на слишком ровное число. Возможно есть какой-то другой порог, который будет более полезен в определении оттока. Возможно, для каждого клиента он свой.

И в итоге с одной стороны можно сравнить эти группы и попробовать найти в них уникальные характеристики. С другой стороны, еще более перспективным вариантом будет рассчитывать критерий оттока по каждому клиенту, опираясь на его характеристики регулярности покупок и как-то так определить таргет.

**Но сейчас нам нужен бейзлайн**, для которого нужны уже готовые таргеты.  
И оттоком определим типы 3,4,5, т.е. оттоком помечается клиент совершивший больше **одной (1)** покупки и соответствует любому из условий: 
* дельта >= 45, 
* прошло больше 45 дней после последней покупки на условный текущий момент.

**Создадим такой таргет и сравним его с изначальным**

1. Создадим фичу с дельтой в днях между покупками
2. Создадим фичу с условием - дельта >= 45?
3. Нужно создать фичу с условием "единственная ли это покупка"  

    a. Создадим признак с номером покупки  
    b. Создадим признак с максимальным номером покупки   
    c. Создадим фичу, в которой сравниваем равен ли номер покупки с максимальным номером покупки этого юзера - это и есть "единственная ли это покупка"  
    
4. Создадим фичу, которая будет говорить последня ли это покупка, если всего покупок больше 1
5. Создадим фичу, которая отсчитывает 45 дней назад с гипотетического сейчас (в нашем случае это будет 2021-12-31)
6. Создадим таргет, который будет соответствовать условиям (**ИЛИ**):

    a. Дельта >= 45  
    b. Последняя покупка из серии покупок **И** прошло больше 45 на гипотетический текущий момент

<!-- **Действительно ли таргет 1 там, где дельта между датами покупки >= 45 дней?**  
Для этого создадим соответствующее условие, применим его к данным со сделками и сравним в таргетом.
1. Создадим фичу с дельтой в днях между покупками по каждому клиенту
2. Проверим, если максимальная дельта у клиента больше или равна 45 дням, то ставим флаг 1, в противном случае 0.
3. Сравниваем получившиеся флаги с предоставленными данными по таргету. -->

<!-- <div class="alert alert-block alert-info">
<b>Комментарий студента: </b> Вот здесь мне пока не до конца ясно, что именно мы считаем. По каким условиям определяется таргет.  
    
* Я сначала сделал просто по одному условию, что дельта в днях между покупками >= 45 дням.  
* Потом добавил еще условие, в котором те, кто сделал только одну покупку, они тоже бы флаговались. - но просто написал код, описание не давал.  
* Потом понял, что бывают клиенты с несколькими покупками, между которыми расстояние меньше 45 дней, но с последней покупки они не делали новой в течении 45 дней - это тоже нужно учесть. Для этого пока код не написал.  

* В конечном счете надо точно разобраться, что именно заказчик ожидает от таргета, что именно подразумевает под оттоком и какую логику флагования действительно стоит использовать.*
</div> -->

In [44]:
# # # Проверяем как работает групбай.
# # # Надо убедиться, что он сохраняет порядок дат.
# # # В документации написано, что сохраняет, но нужно убедиться
# # # Groupby preserves the order of rows within each group. - from pandas documentation

# def time_delta_sorted(col):
#     return col.sort_values().diff()

# def time_delta_as_is(col):
#     return col.diff()

# time_delta_sorted = df.groupby('clnt_ID')['timestamp'].transform(time_delta_sorted)
# time_delta_as_is = df.groupby('clnt_ID')['timestamp'].transform(time_delta_as_is)

# display(time_delta_sorted.compare(time_delta_as_is))
# # # Разница есть

# # Возьмем одного клиента из того индекса, где несовпадение .loc[2023446]

# df_test = df.copy()
# df_test['time_delta_sorted'] = time_delta_sorted
# df_test['time_delta_as_is'] = time_delta_as_is

# display(df_test.loc[2023446])

# # Это AE47EEE1-D3CA-4F80-967B-949F2228192B
# test_client = df_test[df_test['clnt_ID'] == 'AE47EEE1-D3CA-4F80-967B-949F2228192B']

# # Проверим отсортирован ли он по дате (даже после сортировки всего датафрейма)
# test_client.reset_index(drop=True).compare(test_client.sort_values('timestamp').reset_index(drop=True))

# test_client.iloc[93:97,]

# # Отсортируем его по дате
# test_client = test_client.sort_values(by='timestamp')

# test_client.iloc[93:97,]

# # Эти строки поменялись местами

# # Еще раз отсортируем

# test_client = test_client.sort_values(by='timestamp')
# test_client.iloc[93:97,]

# # Они снова поменялись местами

# # Создадим реальный diff
# test_client['real_timestamp_diff'] = test_client['timestamp'].diff()

# # Сравним с трансформом с сортировкой - столбец 'time_delta_sorted'
# test_client['real_timestamp_diff'].compare(test_client['time_delta_sorted'])

# # Сравним с трансформом без сортировки - столбец 'time_delta_as_is'
# test_client['real_timestamp_diff'].compare(test_client['time_delta_as_is'])

# # >>>>>>>>>>>ВЫВОД<<<<<<<<<<<<<<
# # Вывод: Если при сравнени сортированного и несортированного датасета несовпадают только те строки, 
# # в которых идентичное время, то групбай сохраняет последовательность строк. 
# # Проблема лишь в том, как сортировка обрабатывает идентичные значения.
# # Поэтому достаточно лишь раз отсортировать изначальный датасет.

In [ ]:
# Создаем фичу с дельтой в днях между покупками по каждому клиенту
def time_delta(col):
    return col.sort_values().diff()

df['days_delta'] = df.groupby('clnt_ID')['timestamp'].transform(time_delta)
df['days_delta'] = df['days_delta'].dt.days
df['days_delta'] = df['days_delta'].fillna(-1) # Это делаем, чтобы значение "вообще не было покупок ранее" отличалось от "прошло 0 дней с прошлой покупки"

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
# Делаем признак больше или равна ли дельта 45
df['delta>=45'] = df['days_delta'] >= 45

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df[df['delta>=45'] == True].head()

In [ ]:
df[df['delta>=45'] == True].tail()

In [ ]:
# Нам нужно понять единственная ли это покупка
# И последняя ли это покупка, чтобы посмотреть прошло ли с последней покупки 45 дней

# Создадим признак с номером покупки
def expand_count(col):
    return col.sort_values().expanding().count()

df['clnt_buys_count'] = df.groupby('clnt_ID')['timestamp'].transform(expand_count)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
# Нам нужно понять единственная ли это покупка 2

# Для этого создадим столбец с максимальным кол-вом покупок клиента
df['max_buys_count'] = df.groupby('clnt_ID')['clnt_buys_count'].transform('max')

# Если он равен 1, то покупок было всего 1
df['just_1_buy'] = df['max_buys_count'] == 1

In [ ]:
df[df['max_buys_count'] == 1]

In [ ]:
# Здесь нужно понять последняя ли покупка текущая и не единственная ли она
df['last_buy_out_of_more_than_1'] = (df['just_1_buy'] == False)&(df['clnt_buys_count'] == df['max_buys_count'])

In [ ]:
df[df['clnt_buys_count'] == df['max_buys_count']]

In [ ]:
# Теперь представим, что мы в конце года. 2021-12-31
# И отсечем все сделки, которые произошли на текущий день минус 45 дней.
hypothetical_now = pd.Timestamp('2021-12-31')
hypothetical_45_days_before = hypothetical_now - pd.Timedelta(value=45,unit='days')
df['more_than_45_till_now'] = df['timestamp'] < hypothetical_45_days_before

In [ ]:
hypothetical_45_days_before

In [ ]:
df[~(df['timestamp'] < hypothetical_45_days_before)]

In [ ]:
# Теперь делаем таргет
# Ставим 1 в тех сделках, в которых дельта >= 45
# И в тех, в которых это последняя покупка и прошло больше 45 дней (на условный текущий момент)
df_target_extracted = df['delta>=45'] | (df['last_buy_out_of_more_than_1']&df['more_than_45_till_now'])
df_target_extracted.name = 'target'

Посмотрим на распределение таргета в текущем датасете (по сделкам).

In [ ]:
pd.concat([df_target_extracted.value_counts(),df_target_extracted.value_counts(normalize=True)],axis=1)

Посмотрим на баланс таргета по клиентам.

In [ ]:
temp_df = pd.concat([df.iloc[:,0],df_target_extracted],axis=1).groupby('clnt_ID').max()
pd.concat([temp_df.value_counts(),temp_df.value_counts(normalize=True)],axis=1)

И сравним его с таргетом заказчика.

In [ ]:
# Посмотрим и сравним с таргетом, который был даден заказчиком
pd.concat([df_target['target'].value_counts(),df_target['target'].value_counts(normalize=True)],axis=1)

Есть разница:  
В моем скрипте в оттоке 118707 (True),  
В предоставленных данных 126252.

**Есть ли дисбаланс классов?**  
Дисбаланс по сделкам существенный: 5% всего относится к оттоку  
Дисбаланс по клиентам несущественный: 54% на 46% (отток)

**Будем использовать только новый таргет**

## Поиск аномалий в данных

Для поиска аномалий воспользуемся собственной функцией подсчета статистик числовых признаков (но не даты).

In [ ]:
def describe(col):
    """
    Функция выводит подробное числовое описание данных.
    """

#     Подсчет количества значений
    count_ = col.count()
    
#     Меры центральной тенденции:
#     Среднее
    mean = col.mean()
#     Медиана
    median = col.median()
#     Мода
    mode = col.mode()

#     Меры разброса
#     Дисперсия
    var = col.var()
#     Стандартное отклонение
    stddev = col.std()
#     Межквартильный размах
    quartile_1 = col.quantile(0.25)
    quartile_3 = col.quantile(0.75)
    iqr = quartile_3 - quartile_1
#     Мин, макс, размах
    min_ = col.min()
    max_ = col.max()
    range_ = max_ - min_

#     Левая граница ящика с усами
    outlier_border_left = quartile_1 - 1.5 * iqr
#     Правая граница ящика с усами
    outlier_border_right = quartile_3 + 1.5 * iqr
    
#    Количество выбросов
    outliers_mask = (col <= outlier_border_left) | (col >= outlier_border_right)
    outliers_count = col.loc[outliers_mask].count()
    outliers_proportion = outliers_count / count_
    
#     Создание и вывод таблицы с числовым описанием данных
    describe_ = {"count": count_,
                 "mean": mean,
                 "mode": mode,
                 "var": var,
                 "stddev": stddev,
                 "min": min_,
                 "25%": quartile_1,
                 "50%": median,
                 "75%": quartile_3,
                 "max": max_,
                 "total_range": range_,
                 "interquartile_range": iqr,
                 "outlier_border_left": outlier_border_left,
                 "outlier_border_right": outlier_border_right,
                 "outliers_count": outliers_count,
                 "outliers_proportion": outliers_proportion}
    describe_ = pd.DataFrame(describe_).transpose()
    describe_.columns = [col.name]
    return describe_

# describe(t_0)

**timestamp**

In [ ]:
df['timestamp'].describe(datetime_is_numeric=True)

In [ ]:
fig,ax = plt.subplots(figsize=(15,8))
sns.histplot(x=df['timestamp'].dt.date,ax=ax)
plt.show()


* Есть пик в районе 2021-05
* "Яма" - 2021-11

**gest_Sum**

In [ ]:
describe(df['gest_Sum'])

* Есть 0. Что они значат?
* Есть ~6% выборосов.

Сколько нулей?

In [ ]:
df['gest_Sum'].value_counts().sort_index()[0:10]

In [ ]:
gest_Sum_zeros_count = df['gest_Sum'].value_counts().sort_index()[0]
print(f'Нулей в "gest_Sum": {gest_Sum_zeros_count}')

In [ ]:
fig,ax = plt.subplots(2,1,figsize=(15,8))
sns.histplot(x=df['gest_Sum'],ax=ax[0],binwidth=10)
sns.boxplot(x=df['gest_Sum'],ax=ax[1])
# ax.set_xlim(0,1200)
plt.show()
# ДОБАВИТЬ НАЗВАНИЕ ГРАФИКОВ И ОСЕЙ

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(15,5))
sns.histplot(x=df['gest_Sum'],ax=ax,binwidth=10,binrange=(0,1500))

plt.show()
# ДОБАВИТЬ НАЗВАНИЕ ГРАФИКОВ И ОСЕЙ

* Есть пик в районе 100 руб. Кофе?

**gest_Discount**

In [ ]:
describe(df['gest_Discount'])

* Почти 10% выбросов.

In [ ]:
fig,ax = plt.subplots(2,1,figsize=(15,8))
sns.histplot(x=df['gest_Discount'],ax=ax[0],binwidth=1)
sns.boxplot(x=df['gest_Discount'],ax=ax[1])
# ax.set_xlim(0,1200)
plt.show()
# ДОБАВИТЬ НАЗВАНИЕ ГРАФИКОВ И ОСЕЙ

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(15,5))
sns.histplot(x=df['gest_Discount'],ax=ax,binwidth=1,binrange=(0,100))

plt.show()
# ДОБАВИТЬ НАЗВАНИЕ ГРАФИКОВ И ОСЕЙ

Посчитаем кол-во скидок равных 0.

In [ ]:
pd.concat([df['gest_Discount'].value_counts().iloc[0:5],df['gest_Discount'].value_counts(normalize=True).iloc[0:5]],axis=1)

In [ ]:
gest_Discount_zeros_count = df['gest_Discount'].value_counts()[0]
print(f'Кол-во нулей в "gest_Discount": {gest_Discount_zeros_count}')
gest_Discount_zeros_count = df['gest_Discount'].value_counts(normalize=True)[0]
print(f'Процентов нулей в "gest_Discount" от общего числа: {gest_Discount_zeros_count:.02%}')

## Сравнение признаков с разделение по целевому классу

**Сравнение по стоимости покупки за вычетом скидки 'gest_Sum'**

In [ ]:
# df_eda_orig = df.merge(df_target,on='clnt_ID')
df_eda_new = df.merge(df_target_extracted,on='clnt_ID')
# display(df_eda_orig.head())
df_eda_new.head()

In [ ]:
# # По оригинальному таргету
# fig, ax = plt.subplots(1, 1, figsize=(10, 4))
# sns.boxplot(x='gest_Sum', y='target', data=df_eda_orig, ax=ax,orient='h')
# ax.set_xlim(0,1500)
# plt.show()
# # ДОБАВИТЬ НАЗВАНИЕ ГРАФИКОВ И ОСЕЙ

In [ ]:
# По новому таргету
fig, ax = plt.subplots(1, 1, figsize=(10, 4))
sns.boxplot(x='gest_Sum', y='target_ext', data=df_eda_new, ax=ax,orient='h')
ax.set_xlim(0,1500)
plt.show()
# ДОБАВИТЬ НАЗВАНИЕ ГРАФИКОВ И ОСЕЙ

* Напоминает разовые крупные покупки. Типа в день рождения купить большой торт - это редкие клиенты.
* А те, кто в отток не попадает, покупает переодично и поменьше.

**Сравнение по скидке 'gest_Discount'**

In [ ]:
# # По оригинальному таргету
# fig, ax = plt.subplots(1, 1, figsize=(10, 4))
# sns.boxplot(x='gest_Discount', y='target', data=df_eda_orig, ax=ax,orient='h')
# ax.set_xlim(0,200)
# plt.show()
# # ДОБАВИТЬ НАЗВАНИЕ ГРАФИКОВ И ОСЕЙ

In [ ]:
# По новому таргету
fig, ax = plt.subplots(1, 1, figsize=(10, 4))
sns.boxplot(x='gest_Discount', y='target_ext', data=df_eda_new, ax=ax,orient='h')
ax.set_xlim(0,200)
plt.show()
# ДОБАВИТЬ НАЗВАНИЕ ГРАФИКОВ И ОСЕЙ

* У тех, кто остается есть какая-то скидка
* У тех, кто уходит в отток - никакой скидки нет.

**Сравнение по общей стоимости чека ('gest_Sum'+'gest_Discount')**

Здесь нужно создать столбец с общей стоимостью чека.

In [ ]:
# df_eda_orig['gest_Total'] = df_eda_orig['gest_Sum'] + df_eda_orig['gest_Discount'] 
# df_eda_orig.head()

In [ ]:
df_eda_new['gest_Total'] = df_eda_new['gest_Sum'] + df_eda_new['gest_Discount']
df_eda_new.head()

In [ ]:
# # По оригинальному таргету
# fig, ax = plt.subplots(1, 1, figsize=(10, 4))
# sns.boxplot(x='gest_Total', y='target', data=df_eda_orig, ax=ax,orient='h')
# ax.set_xlim(0,1500)
# plt.show()
# # ДОБАВИТЬ НАЗВАНИЕ ГРАФИКОВ И ОСЕЙ

In [ ]:
# По новому таргету
fig, ax = plt.subplots(1, 1, figsize=(10, 4))
sns.boxplot(x='gest_Total', y='target_ext', data=df_eda_new, ax=ax,orient='h')
ax.set_xlim(0,1500)
plt.show()
# ДОБАВИТЬ НАЗВАНИЕ ГРАФИКОВ И ОСЕЙ

* Здесь боксплот говорит примерно о том же, что в gest_Sum выше.

**Сравнение зависимости gest_Sum от gest_Discount с разделением по таргету**

In [ ]:
# sns.relplot(x="gest_Sum", y="gest_Discount", hue="target", data=df_eda_orig);
# # ДОБАВИТЬ НАЗВАНИЕ ГРАФИКОВ И ОСЕЙ

Похоже на то, что есть система фиксированных скидок и есть накопительные баллы или акции. Прямые линии это фиксированные скидки.

In [ ]:
sns.relplot(x="gest_Sum", y="gest_Discount", hue="target_ext", data=df_eda_new);
# ДОБАВИТЬ НАЗВАНИЕ ГРАФИКОВ И ОСЕЙ

* Видим много оттока при скидке = 0
* Видим какую-то акцию, где скидка была 1000 руб.
* Видим оставшихся с фиксированной скидкой (линия больше 45 градусов)

**Корреляция**

In [ ]:
# orig_corr = df_eda_orig.corr()
new_corr = df_eda_new.corr()

In [ ]:
# fig,ax = plt.subplots(figsize=(7,7))
# sns.heatmap(orig_corr, annot=True,ax=ax,vmin=-1,vmax=1,linewidth=.5,cmap="PiYG",center=0,fmt='.2%')
# plt.show()

In [ ]:
fig,ax = plt.subplots(figsize=(7,7))
sns.heatmap(new_corr, annot=True,ax=ax,vmin=-1,vmax=1,linewidth=.5,cmap="PiYG",center=0,fmt='.2%')
plt.show()

Чего-то особого, кроме очевидного, здесь не обнаружено.

# Feature engineering

Уже созданы:
* Дельта в днях между ближайшими сделками (-1, если сделка первая) (Исследовать)
* Флаг дельта >= 45 (Исследовать)
* Номер покупки клиента
* Максимальный номер покупки клиента
* Флаг "Клиент сделал всего 1 покупку за всю историю"
* Флаг "Последняя сделка клиента в этих данных при условии, что покупок было больше 1"

Новые фичи.  
Что может помочь лучше предсказать отток клиентов? --ДОПОЛНЯЕТСЯ--

* Общая стоимость заказа без скидки
* Сколько дней на момент сделки прошло с момента первой сделки (сколько дней клиенту)
* Какой на текущий момент средний чек, средняя скидка, средний общая стоимость
* Какая разница между среднем чеком и чеком сделки
* Сколько сделок произошло на момент текущей сделки
* Сколько сделок в месяц в среднем


**Как создавать сложные фичи?**

* Нужно группировать датасет по клиентам
* К этим группам применять разные методы с помощью `.groupby().transform()`

`df.groupby('cnlt_ID').transform(func)`

**Общая стоимость заказа без скидки** = `gest_Sum + gest_Discount`

In [ ]:
df['gest_Total'] = df['gest_Sum'] + df['gest_Discount']

**Сколько дней прошло с первой сделки**

1. Берем min(date)
2. Вычитаем из текущей data min(date) (`df['date'] - df['first_buy_date']`)

In [ ]:
def first_buy_date_delta(col):
    return col - col.min()

df['first_buy_days_delta'] = df.groupby('clnt_ID')['timestamp'].transform(first_buy_date_delta).dt.days

In [ ]:
df.tail()

**TO DO: ИЗУЧИТЬ**

In [ ]:
# df['first_buy_date_delta'].dt.days.astype('int').hist(bins=100)

In [ ]:
# df['first_buy_timestamp_delta'].dt.total_seconds().astype('int').hist(bins=100)

**Какой на текущий момент средний чек, средняя скидка, средний общая стоимость**

Используем `series.expanding().mean()` через `df.groupby('clnt_ID').transform()`.

In [ ]:
def expand_mean(col):
    return col.expanding().mean()

df['clnt_gest_Sum_avg_ongoing'] = df.groupby('clnt_ID')['gest_Sum'].transform(expand_mean)
df['clnt_gest_Discount_avg_ongoing'] = df.groupby('clnt_ID')['gest_Discount'].transform(expand_mean)
df['clnt_gest_Total_avg_ongoing'] = df.groupby('clnt_ID')['gest_Total'].transform(expand_mean)

In [ ]:
df.head()

**TO DO: ИЗУЧИТЬ**

In [ ]:
# df['clnt_gest_Sum_avg'].hist(bins=100)

**Какая разница между средним значением и значением сделки**
Считаем для:
* 'gest_Sum'
* 'gest_Discount'
* 'gest_Total'

`df['gest_Sum'] - df['clnt_gest_Sum_avg_ongoing']`  
`df['gest_Discount'] - df['clnt_gest_Discount_avg_ongoing']`  
`df['gest_Total'] - df['clnt_gest_total_avg_ongoing']`

In [ ]:
df['gest_Sum_delta_from_avg'] = df['gest_Sum'] - df['clnt_gest_Sum_avg_ongoing']
df['gest_Discount_delta_from_avg'] = df['gest_Discount'] - df['clnt_gest_Discount_avg_ongoing']
df['gest_total_delta_from_avg'] = df['gest_Total'] - df['clnt_gest_Total_avg_ongoing']

In [ ]:
df.tail()

**TO DO: ИЗУЧИТЬ**

In [ ]:
# df['gest_total_delta_from_avg'].hist(bins=100)

**Сколько сделок произошло на момент текущей сделки**  
`series.expanding().count()`

In [ ]:
def expand_count(col):
    return col.expanding().count()

df['clnt_buys_count'] = df.groupby('clnt_ID')['timestamp'].transform(expand_count)

In [ ]:
df.tail()

**TO DO: ИЗУЧИТЬ**

In [ ]:
# df['clnt_buys_count'].describe()

In [ ]:
# df['clnt_buys_count'].hist(bins=100)

# BASELINE и model selection

<div class="alert alert-block alert-info">
<b>Комментарий студента: </b> Здесь у меня пока возникает оверфит.
    
Первое предположение было о том, что кол-во дней между покупками фактически описывает таргет. Убрал эту фичу совсем, но оверфит остался.
Теперь думаю попробовать следующее:
* Посчитать статистики не за всю историю, а за последние 30 дней или около того. (Распределение кол-ва дней между покупками может подсказать порог в днях получше. Может больше 30, может меньше, но не более 45)
* Использовать не агрегированный по юзерам датасет, а изначальный по сделкам и добавить в каждой сделке накопительную статистику истории пользователя. 
</div>

Для бейслайна используем данные в таком виде:
* Группируем по клиенту,
* берем общее число покупок
* мин, макс, медиана, среднее, стд отклонение по `gest_Sum`, `gest_Discount`, 
* ~~кол-во дней между покупками, медиана, среднее, стд. отклонение.~~ Кол-во дней между покупками использовать нельзя, потому что это утечка. По среднему и стандартному отклонению можно точно определить был ли хоть раз промежуток в 45 дней между покупками. (И у меня был тут скор ~0.98 из-за этих фичей)

Модели:
* логистическая регрессия
* дерево решений
* случайный лес

Оцениваем `roc auc` по кроссвалидации на 5 фолдах.

Проверять будем по новому таргету.

In [ ]:
# Скопируем датасет
df_baseline = df.copy()
df_baseline = df_baseline.drop(columns=['timestamp'])
df_baseline.head()

In [ ]:
# Создадим словарь для функция агрегирования по необходимым столбцам
aggfunc_dict = {'gest_Sum': ['min','max','median','mean',
                             # 'std'
                            ], 
                'gest_Discount': ['min','max','median','mean',
                                  # 'std'
                                 ], 
                # 'clnt_buys_count': ['max','median','mean','std'] # здесь min не нужен, т.к. везде будут 1
               } 

In [ ]:
# Сгруппируем по клиентам применим агрегирующие функции из словаря
pivot_baseline = df_baseline.pivot_table(index='clnt_ID', values=['gest_Sum','gest_Discount','clnt_buys_count'],aggfunc=aggfunc_dict, 
                                         fill_value=0 # Пропуски будут в стандартном отклонении любых признаков, где только 1 покупка
                                        )

In [ ]:
# Уберем мульти-уровни в столбцах
pivot_baseline.columns = ['_'.join(column) for column in pivot_baseline.columns]

In [ ]:
pivot_baseline.describe()

In [ ]:
# Создадим датафрейм с присоединенными таргетами
# pivot_baseline_full_orig = pivot_baseline.merge(df_target,on='clnt_ID')
pivot_baseline_full_new = pivot_baseline.merge(df_target_extracted,on='clnt_ID').reset_index()

In [ ]:
# pivot_baseline_full_orig.head()

In [ ]:
pivot_baseline_full_new.head()

In [ ]:
# Уберем айдишники клиентов. Модели они только помешают
# pivot_baseline_full_orig = pivot_baseline_full_orig.drop(columns=['clnt_ID'])
pivot_baseline_full_new = pivot_baseline_full_new.drop(columns=['clnt_ID'])


In [ ]:
# Создадим фичи и таргет

# # Оригинальный
# X_orig = pivot_baseline_full_orig.drop(columns=['target'])
# y_orig = pivot_baseline_full_orig['target']

# Новый
X_new = pivot_baseline_full_new.drop(columns=['target_ext'])
y_new = pivot_baseline_full_new['target_ext']

In [ ]:
# display(X_orig.head())
# y_orig.head()

In [ ]:
display(X_new.head())
y_new.head()

In [ ]:
# Логистическая регрессия
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    log_reg = LogisticRegression()
    # log_reg_cv_orig = cross_validate(log_reg, X_orig, y_orig, cv=5, scoring='roc_auc')
    log_reg_cv_new = cross_validate(log_reg, X_new, y_new, cv=5, scoring='roc_auc')
    
# Здесь ошибки несходимости - убрал

In [ ]:
# Дерево решений
tree = DecisionTreeClassifier()
# tree_cv_orig = cross_validate(tree,X_orig,y_orig,cv=5,scoring='roc_auc')
tree_cv_new = cross_validate(tree,X_new,y_new,cv=5,scoring='roc_auc')

In [ ]:
# Случайный лес
forest = RandomForestClassifier()
# forest_cv_orig = cross_validate(forest,X_orig,y_orig,cv=5,scoring='roc_auc')
forest_cv_new = cross_validate(forest,X_new,y_new,cv=5,scoring='roc_auc')

In [ ]:
# Подсчет и вывод метрик

# # Оригинальный
# lr_roc_auc_orig = log_reg_cv_orig['test_score'].mean()
# tree_roc_auc_orig = tree_cv_orig['test_score'].mean()
# forest_roc_auc_orig = forest_cv_orig['test_score'].mean()

# Новый
lr_roc_auc_new = log_reg_cv_new['test_score'].mean()
tree_roc_auc_new = tree_cv_new['test_score'].mean()
forest_roc_auc_new = forest_cv_new['test_score'].mean()

scores = {
        # 'ORIG logreg roc auc': lr_roc_auc_orig,
        #  'ORIG tree roc auc': tree_roc_auc_orig,
        #  'ORIG forest roc auc': forest_roc_auc_orig,
         'NEW logreg roc auc': lr_roc_auc_new,
         'NEW tree roc auc': tree_roc_auc_new,
         'NEW forest roc auc': forest_roc_auc_new}

for key,value in scores.items():
    print(f'{key} {value:.03f}')

Подозрительно высокий скор на всех моделях.

In [ ]:
# with warnings.catch_warnings():
#     warnings.simplefilter('ignore')
#     log_reg = LogisticRegression().fit(X_new,y_new)
# log_reg.coef_

In [ ]:
forest = RandomForestClassifier(random_state=rand_state).fit(X_new,y_new)

In [ ]:
list(zip(X_new.columns,forest.feature_importances_))

In [ ]:
plt.bar(x=X_new.columns,height=forest.feature_importances_)
plt.xticks(rotation=90)
plt.show()

### catboost -- ON HOLD

In [ ]:
# ON HOLD

# test_data = catboost_pool = Pool(train_data, 
#                                  train_labels)

# model = CatBoostClassifier(iterations=2,
#                            depth=2,
#                            learning_rate=1,
#                            loss_function='Logloss',
#                            verbose=True)
# # train the model
# model.fit(train_data, train_labels)
# # make the prediction using the resulting model
# preds_class = model.predict(test_data)
# preds_proba = model.predict_proba(test_data)
# print("class = ", preds_class)
# print("proba = ", preds_proba)

# Model tuning

EMPTY